In [25]:
import numpy as np
from scipy.ndimage import label, center_of_mass
from collections import defaultdict

GLOBAL_MAP = """WWWWWWWWWWWWWWWWWWWWWWWW
WAAA    A      A    AAAW
WAA    AAA    AAA    AAW
WA    AAGAA  AAAAA    AW
W      AAA    AAA      W
W       A      A       W
W  A                A  W
W AAA  Q        Q  AAA W
WAAAAA            AAAAAW
W AAA              AAA W
W  A                A  W
W                      W
W                      W
W                      W
W  PPPPPPPPPPPPPPPPPP  W
W PPPPPPPPPPPPPPPPPPPP W
WPPPPPPPPPPPPPPPPPPPPPPW
WWWWWWWWWWWWWWWWWWWWWWWW"""



def connected_elems_map(ascci_map, elements_to_find):

    # Convierte la matriz en una matriz numpy
    matrix = np.array([list(row) for row in ascci_map.split('\n')])

    # Generate mask
    mask = (matrix == elements_to_find[0]) 
    for elem in elements_to_find[1:]:
        mask |= (matrix == elem)

    
    # Encontrar componentes conectados
    labeled_matrix, num_features = label(mask)

    # Inicializa un diccionario para almacenar los centros de los componentes y sus elementos
    component_data = defaultdict(list)

    # Calcula el centro de cada componente y almacena sus elementos
    for i in range(1, num_features + 1):
        component_mask = labeled_matrix == i
        center = center_of_mass(component_mask)
        center_coords = (int(center[0]), int(center[1]))
        component_elements = np.argwhere(component_mask)
        component_data[i] = {'center': center_coords, 'elements': component_elements.tolist()}

    return dict(component_data)

global_elements = connected_elems_map(GLOBAL_MAP, elements_to_find=['A', 'G'])
global_elements

{1: {'center': (1, 1),
  'elements': [[1, 1], [1, 2], [1, 3], [2, 1], [2, 2], [3, 1]]},
 2: {'center': (3, 8),
  'elements': [[1, 8],
   [2, 7],
   [2, 8],
   [2, 9],
   [3, 6],
   [3, 7],
   [3, 8],
   [3, 9],
   [3, 10],
   [4, 7],
   [4, 8],
   [4, 9],
   [5, 8]]},
 3: {'center': (3, 15),
  'elements': [[1, 15],
   [2, 14],
   [2, 15],
   [2, 16],
   [3, 13],
   [3, 14],
   [3, 15],
   [3, 16],
   [3, 17],
   [4, 14],
   [4, 15],
   [4, 16],
   [5, 15]]},
 4: {'center': (1, 21),
  'elements': [[1, 20], [1, 21], [1, 22], [2, 21], [2, 22], [3, 22]]},
 5: {'center': (8, 3),
  'elements': [[6, 3],
   [7, 2],
   [7, 3],
   [7, 4],
   [8, 1],
   [8, 2],
   [8, 3],
   [8, 4],
   [8, 5],
   [9, 2],
   [9, 3],
   [9, 4],
   [10, 3]]},
 6: {'center': (8, 20),
  'elements': [[6, 20],
   [7, 19],
   [7, 20],
   [7, 21],
   [8, 18],
   [8, 19],
   [8, 20],
   [8, 21],
   [8, 22],
   [9, 19],
   [9, 20],
   [9, 21],
   [10, 20]]}}

In [26]:

# Define la matriz de caracteres ASCII
global_position = (7,16)
local_position = (9,5)
LOCAL_MAP = """-----------
-----------
WWWWWWWWWWW
FFFFAFFFFAA
FFFAAAFFFFA
FFAAGAAFFFF
FFFAAAFFFFF
FFFFAFFFFFF
F1FFFFFFFAF
FFFFF#FFAAA
FFFFFFFAAAA"""


list_of_observations = []

def get_element_global_pos(element_local, local_position, global_position):
    element_global = (element_local[0] - local_position[0]) + global_position[0],\
                         (element_local[1] - local_position[1]) + global_position[1]
    return list(element_global)



def get_trees_descriptions(local_map, local_position, global_position):
    local_tree_elements = connected_elems_map(local_map, elements_to_find=['A', 'G'])
    list_trees_observations = []
    trees_observed = []
    for _, local_tree_data in local_tree_elements.items():
        local_tree_center = local_tree_data['center']
        local_center_real_pos = get_element_global_pos(local_tree_center, local_position, global_position)
        print(local_center_real_pos)
        for global_tree_id, global_tree_data in global_elements.items():

            # Continue if the tree has already been observed
            if global_tree_id in trees_observed: 
                continue

            # Find the cluster tree of the local tree
            if local_center_real_pos in global_tree_data['elements']:
                trees_observed.append(global_tree_id)
                print(trees_observed)
                apple_count, grass_count = 0, 0
                for apple in local_tree_data['elements']:
                    apple_global_pos = get_element_global_pos(apple, local_position, global_position)
                    if LOCAL_MAP.split('\n')[apple[0]][apple[1]] == 'G':
                        list_trees_observations.append("Observed grass to grow apples at position {}. This grass belongs to tree {}"
                                                    .format(apple_global_pos, global_tree_id))
                        grass_count += 1
                    else:
                        list_trees_observations.append("Observed an apple at position {}. This apple belongs to tree {}"
                                                    .format(apple_global_pos, global_tree_id ))
                        apple_count += 1
                
                list_trees_observations.append("Observed tree {} at position {}. This tree has {} apples remainding and {} grass for apples growing"
                                            .format(global_tree_id, local_center_real_pos, apple_count, grass_count))
                break
    return list_trees_observations

list_of_observations.extend(get_trees_descriptions(LOCAL_MAP, local_position, global_position))
list_of_observations

[3, 15]
[3]
[1, 20]
[3, 4]
[7, 19]
[3, 4, 6]


['Observed an apple at position [1, 15]. This apple belongs to tree 3',
 'Observed an apple at position [2, 14]. This apple belongs to tree 3',
 'Observed an apple at position [2, 15]. This apple belongs to tree 3',
 'Observed an apple at position [2, 16]. This apple belongs to tree 3',
 'Observed an apple at position [3, 13]. This apple belongs to tree 3',
 'Observed an apple at position [3, 14]. This apple belongs to tree 3',
 'Observed grass to grow apples at position [3, 15]. This grass belongs to tree 3',
 'Observed an apple at position [3, 16]. This apple belongs to tree 3',
 'Observed an apple at position [3, 17]. This apple belongs to tree 3',
 'Observed an apple at position [4, 14]. This apple belongs to tree 3',
 'Observed an apple at position [4, 15]. This apple belongs to tree 3',
 'Observed an apple at position [4, 16]. This apple belongs to tree 3',
 'Observed an apple at position [5, 15]. This apple belongs to tree 3',
 'Observed tree 3 at position [3, 15]. This tree has